# UB Herbarium Case Study

In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# Reading the dataset

In [2]:
dsetPath = '/home/pedro/datasets/ub_herbarium/occurrence.txt'

In [3]:
cols = ['recordedBy', 'scientificName', 'family', 'genus', 'species','taxonRank', 
        'stateProvince', 'locality', 'municipality', 'occurrenceRemarks',
        'eventDate', 'identifiedBy']
occs = pd.read_table(dsetPath, delimiter='\t', usecols=cols, low_memory=False)
occs.dropna(subset=['recordedBy','scientificName'], inplace=True)

In [4]:
occs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185301 entries, 0 to 185310
Data columns (total 12 columns):
recordedBy           185301 non-null object
occurrenceRemarks    140768 non-null object
eventDate            181254 non-null object
stateProvince        174285 non-null object
municipality         135291 non-null object
locality             163029 non-null object
identifiedBy         148600 non-null object
scientificName       185301 non-null object
family               182987 non-null object
genus                176755 non-null object
taxonRank            185301 non-null object
species              152360 non-null object
dtypes: object(12)
memory usage: 18.4+ MB


In [5]:
occs.head()

,recordedBy,occurrenceRemarks,eventDate,stateProvince,municipality,locality,identifiedBy,scientificName,family,genus,taxonRank,species
0,"Irwin, HS","Ascending subshrub 0,3m. Fruit gray-green; Cer...",1972-01-16T01:00Z,Goiás,NaN,"Serra dos Pireneus, ca. 20km E of Pirenópolis",NaN,Annona monticola Mart.,Annonaceae,Annona,SPECIES,Annona monticola
1,"Ratter, JA; et al.",Coppice branches sprouting from the stump of a...,1976-06-30T01:00Z,Minas Gerais,NaN,"Near Pandeiros, ca. 30.0 km W of Januária",Flora do Brasil,Myracrodruon urundeuva Allem.,Anacardiaceae,Myracrodruon,SPECIES,Myracrodruon urundeuva
2,"Heringer, EP",NaN,1954-06-05T01:00Z,Minas Gerais,NaN,Fazenda do Rasgão. Terreno de cultura,Flora do Brasil,Myracrodruon urundeuva Allem.,Anacardiaceae,Myracrodruon,SPECIES,Myracrodruon urundeuva
3,"Coelho, JP","Pouco frequente, sem folhas nesse período; Árv...",1964-10-15T01:00Z,Minas Gerais,NaN,IPEACO- Sete lagoas,Flora do Brasil,Myracrodruon urundeuva Allem.,Anacardiaceae,Myracrodruon,SPECIES,Myracrodruon urundeuva
4,"Eiten, G; Eiten, LT","Tree 5.0 m. tall, 11.0 cm d.b.h. With young fr...",1963-08-17T01:00Z,Maranhão,Loreto,"Ilha de Balsas region, between the Balsas and ...",Flora do Brasil,Myracrodruon urundeuva Allem.,Anacardiaceae,Myracrodruon,SPECIES,Myracrodruon urundeuva


In [6]:
occs.shape[0]

185301

In [7]:
counts = pd.DataFrame(occs['taxonRank'].value_counts())
counts.columns = ['Counts']
counts['Perc'] = counts['Counts']/counts['Counts'].sum()
counts

,Counts,Perc
SPECIES,140763,0.759645
GENUS,24397,0.131661
VARIETY,8935,0.048219
FAMILY,6223,0.033583
KINGDOM,2008,0.010836
SUBSPECIES,1681,0.009072
FORM,1000,0.005397
PHYLUM,294,0.001587


Cleaning collector data

In [10]:
import paths

In [13]:
from caryocar.cleaning import read_NamesMap_fromJson, NamesAtomizer, namesFromString

# read a names map
nm = read_NamesMap_fromJson('../ub_names_map_2.json')

# atomizing names
atomizingOp = lambda x: namesFromString(x)
na = NamesAtomizer(atomizingOp)

occs['recordedBy_atomized'] = na.atomize(occs['recordedBy'])

/home/pedro/caryocar/caryocar/cleaning.py:536: UserWarning: A names map was created without a normalization function!
  warn("A names map was created without a normalization function!")


Building the names index

In [15]:
from caryocar.cleaning import getNamesIndexes

ni = getNamesIndexes(occs,'recordedBy_atomized',nm.getMap())

# Network Models

In [17]:
from caryocar.models import CoworkingNetwork, SpeciesCollectorsNetwork

In [18]:
namesList = list(set(nm.getMap().values()))

"Total number of collectors in the dataset: "+str(len(namesList)) 

'Total number of collectors in the dataset: 7245'

## Construction

#### Collectors co-working Network

In [19]:
cwn = CoworkingNetwork(namesSets=occs['recordedBy_atomized'], weighted=True, namesMap=nm)

TypeError: 'NoneType' object is not iterable

Filtering some nodes...

In [ ]:
nodesToRemove = ['etal','']
cwn.remove_nodes_from(nodesToRemove)

Adding attributes

In [ ]:
# nodes degrees
nx.set_node_attributes(cwn, 'degree', cwn.degree())

# nodes weighted degrees
nx.set_node_attributes(cwn, 'degree_weighted', cwn.degree(weight='weight'))

# total number of records for each collector (includes non-collaborative records)
count_totalRecords = lambda name,index: len(index[name]) 
nx.set_node_attributes(cwn, 'tot_records', dict( (n, count_totalRecords(n,ni) ) for n in cwn.nodes() ))

In [ ]:
# remove self-loops

self_loops = [ (u,v) for u,v in cwn.edges() if u==v ] 
cwn.remove_edges_from(self_loops)

In [ ]:
nx.write_gexf(cwn, './graphs/ub_graph_cwn.gexf')

#### Species-collectors Network

We'll only use records with the taxonomic resolution of species.

In [ ]:
occs_sp = occs[occs['species'].notnull()]

In [ ]:
scn = SpeciesCollectorsNetwork(species=occs_sp['species'], 
                               collectorsNames=occs_sp['recordedBy_atomized'],
                              weighted=True, namesMap=nm)

Filtering some nodes...

In [ ]:
nodesToRemove = ['etal','']
scn.remove_nodes_from(nodesToRemove)

Adding attributes

In [ ]:
# nodes degrees
nx.set_node_attributes(scn, 'degree', scn.degree())

# nodes weighted degrees
nx.set_node_attributes(scn, 'degree_weighted', scn.degree(weight='weight'))

# set species families
sp_families = occs_sp[['species','family']].drop_duplicates()
sp_families_dict = dict(zip(sp_families['species'],sp_families['family']))
nx.set_node_attributes(scn, 'family', sp_families_dict)

In [ ]:
nx.write_gexf(scn, './ub_graph_scn.gexf')

#### Aggregating species by family

In [ ]:
from collections import Counter

def aggregateByTaxLevel(network, level):
    spNodes = network.getSpeciesNodes(data=True)
    taxonMap = dict( (sp,d[level]) for sp,d in spNodes )
    taxGroupsNames = list(set(taxonMap.values()))

    edgesList = [ (taxonMap[u],v,d) for u,v,d in network.edges((n for n,d in spNodes), data=True) ]
    edges = Counter( (u,v) for u,v,d in edgesList for i in range(d['weight']) )

    net_aggr = SpeciesCollectorsNetwork()
    net_aggr.add_nodes_from(network.getCollectorsNodes(),bipartite=0)
    net_aggr.add_nodes_from(taxGroupsNames, bipartite=1)
    net_aggr.add_edges_from( (u,v,{'weight':w}) for (u,v),w in edges.items() )
    
    return net_aggr

G = aggregateByTaxLevel(scn,'family')

In [ ]:
nx.set_node_attributes(G,'degree',G.degree())
nx.set_node_attributes(G,'degree_weighted',G.degree(weight='weight'))

---

### Comparing collectors species bags

#### Getting a collector's species bag

We can, for example, retrieve the species bag for `proenca,ceb`. When we unpack the tuple, the first element is the list of the species names, and the second is a 1-row sparse matrix where her species bag is stored in:

In [ ]:
i,m = scn.getSpeciesBag('proenca,ceb')

In [ ]:
m.todense()

In [ ]:
# The first ten (species,count) tuples from the speciesBag
list( zip( i, np.array(m.todense()).flatten() ) )[:10]

#### Species Bags Similarity

We calculate the similarity of collectors from their species bag using the Cosine Similarity metric.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import scipy

def calculateSimilarity( G, col1=None, col2=None ):
    """
    Returns
    -------
    If neither col1 and col2 are informed, a tuple ( colsList, csr_sparse_similarityMatrix )
    If only col1 is informed, a tuple ( colsList, similarityVector )
    If col1 and col2 are informed, a tuple ( [col1,col2], similarity )
    """
    colList, spList, m = G._speciesBag_matrix
    
    if col1 is not None:
        if col2 is not None:
            sim = cosine_similarity( scn.getSpeciesBag(col1)[1], scn.getSpeciesBag(col2)[1] )[0][0] 
            return ([col1,col2], sim)
        else:
            simVector = cosine_similarity( scn.getSpeciesBag(col1)[1], m )
            return ( colList, scipy.sparse.csr_matrix(simVector) )
        
    else:
        return( colList, scipy.sparse.csr_matrix(cosine_similarity(m)) )

If we want to retrieve the full species bag matrix, we simply call this function without collectors names:

In [ ]:
i,m = calculateSimilarity(scn)
m

In [ ]:
m.

Alternatively we can get a similarity vector for a particular collector:

In [ ]:
i,m = calculateSimilarity(scn,'munhoz,cbr')
m.todense()

In [ ]:
# These are the top-10 collectors most similar to munhoz,cbr 
sorted(list(zip(i,np.array(m.todense()).flatten())),key=lambda x: x[1], reverse=True)[:10]

We can also get the similarity score for a pair of collectors:

In [ ]:
cols,score = calculateSimilarity(scn,'munhoz,cbr','proenca,ceb')
"Similarity of the species bags of {} and {}: {}".format(*cols,score)

Let's now define a filtering function:

* For a **sparse matrix**, I'll iterate through non-zero elements and apply a threshold function

In [ ]:
i,m = calculateSimilarity(scn)

In [ ]:
m[0:10,164:174].todense()

In [ ]:
thresh = 0.1
m.data = np.where(m.data>=thresh,m.data,0)
m.eliminate_zeros()

In [ ]:
m[0:10,164:174].todense()

* And the same for similarity vectors

In [ ]:
i,m = calculateSimilarity(scn, 'munhoz,cbr')

In [ ]:
m.todense()[0,:10]

In [ ]:
thresh = 0.01
m.data = np.where(m.data>=thresh,m.data,0)
m.eliminate_zeros()

In [ ]:
m.todense()[0,:10]

In [ ]:
list(m.indices)[:10]

In [ ]:
list(m.data)[:10]

In [ ]:
i_row=20
i_col=0
n_rows=10
n_cols=10
'''
for i in range(i_row,i_row+n_rows):
    for j in range(i_col,i_col+n_cols):
        print(i,j)
'''

m[i_row:i_row+n_rows,i_col:i_col+n_cols].todense()

In [ ]:
thresh = lambda x: x if x>0.5 else 0
thresh = np.vectorize(thresh)
m.data = thresh(m.data)

In [ ]:
for i in m.todense()[:10,:10].shape:
    print(i)

In [ ]:
m.todense()[:10,:10].shape

In [ ]:
thresh = lambda x: x if x>0.5 else 0
thresh = np.vectorize(thresh)
m.data = thresh(m.data)

m.

In [ ]:
limit = lambda x: x if x>0.5 else 0
limit = np.vectorize(limit)

m_l = limit(m.todense())

In [ ]:
for i in range(m_l.shape[0]):
    for j in range(m_l.shape[1]):
        print(m[i,j])

In [ ]:
i,m = calculateSimilarity(scn)

limit(m)
#sorted(list(zip(i,m_limit)),key=lambda x: x[1], reverse=True)

In [ ]:
m=cosine_similarity(G._speciesBag_matrix[2])


In [ ]:
col1='moura,co'
col2='siracusa,p'
cosine_similarity(scn.getSpeciesBag(col1),scn.getSpeciesBag(col2))

In [ ]:
m=cosine_similarity(scn._speciesBag_matrix[2])

In [ ]:
thresh = lambda x: x if x>0.6 else 0
thresh=np.vectorize(thresh)

m_t = thresh(m)

In [ ]:
import scipy as scp

m_sparse = scp.sparse.csr_matrix(m_t)


In [ ]:
m_sparse.setdiag(0)

In [ ]:
g = nx.from_scipy_sparse_matrix(m_sparse)

nMap = dict( (i,n) for i,n in enumerate(scn._speciesBag_matrix[0]) )
g = nx.relabel_nodes(g,nMap)

In [ ]:
k_weighted_dict = dict( (n,d['degree_weighted']) for n,d in scn.getCollectorsNodes(data=True) )

In [ ]:
nx.set_node_attributes(g,'k_weighted',k_weighted_dict)

In [ ]:
nx.write_gexf(g,'graph.gexf')

In [ ]:
G_proj = nx.bipartite.projection.collaboration_weighted_projected_graph(G,G.getSpeciesNodes())

In [ ]:
G_proj.edges(data=True)

In [ ]:
G_proj.edges(data=True)

In [ ]:
nx.write_gexf(G_proj,'./ub_graph_scn_family.gexf')

In [ ]:
G = nx.Graph()
G.add_edges_from([('a','b',{'w':1}),('a','c',{'w':2})])

G.edges()

In [ ]:
spNodes = [ (n,d) for n,d in scn.nodes(data=True) if d['bipartite']==1 ]
spDict = dict( (sp,d['family']) for sp,d in spNodes )

In [ ]:
edges = [ (spDict[u],v,d) for u,v,d in scn.edges([n for n,d in spNodes], data=True) ]

In [ ]:
[ (u,[v for i in range(d['weight'])]) for u,v,d in edges ]

In [ ]:
edgesDict = dict()

for u,v,d in edges:
    currWeight = d['weight']
    try:
        edgesDict[(u,v)]['weight'] += currWeight
        
    except:
        edgesDict[(u,v)] = {}
        edgesDict[(u,v)]['weight'] = currWeight

In [ ]:
assert( sum( data['weight'] for k,data in edgesDict.items() ) == sum( data['weight'] for k,v,data in edges ) )

In [ ]:
nx.Graph( (u,v,d) for (u,v),d in edgesDict.items() ).nodes(data=True)

In [ ]:
list(zip(occs['species'],occs['recordedBy_atomized']))

In [ ]:
spNodes = [ (n,d) for n,d in scn.nodes(data=True) if d['bipartite']==1 ]

# First group nodes

groups = [ (data['family'],n) for n,data in spNodes ]
groupsDict = dict()
for k,v in groups:
    groupsDict[k] = [v] + groupsDict.get( k,[] )
    
groupsDict

In [ ]:
# then merge nodes attributes
newNodes = dict()
interest_attrs = ['degree', 'degree_weighted']

for g in groupsDict.keys():
    group_attrs = [ scn.node[n] for n in groupsDict[g] ]

    newNodes[g] = dict( (attr,sum(i[attr] for i in group_attrs)) for attr in interest_attrs )
    
newNodes

In [ ]:
# then merge edges

newEdges = list()

for g in groupsDict.keys():

    neighbors = [ (u,d) for sp in groupsDict[g] for u,d in scn[sp].items() ]

    neighbors_dict =dict()

    for n,data in neighbors:
        neighbors_dict[n] = neighbors_dict.get(n,{'weight':0})
        neighbors_dict[n]['weight'] += data['weight']
        
    newEdges += [ (g,v,data) for v,data in neighbors_dict.items() ]

newEdges

In [ ]:
def aggregateByFamily( network ):
    
    spNodes = [ (n,d) for n,d in scn.nodes(data=True) if d['bipartite']==1 ] # species nodes
    interest_attrs = ['degree', 'degree_weighted'] # node attrs to merge

    # First group nodes
    groups = [ (data['family'],n) for n,data in spNodes ]
    groupsDict = dict()
    for k,v in groups:
        groupsDict[k] = [v] + groupsDict.get( k,[] )
    
    groupsNames = groupsDict.keys()
    
    # then merge nodes attributes
    getNodeAttributes = lambda n: network.node[n]

    newNodes = dict()
    for g in groupsNames:
        group_attrs = [ getNodeAttributes(sp) for sp in groupsDict[g] ]
        newNodes[g] = dict( (attr,sum(i[attr] for i in group_attrs)) for attr in interest_attrs )
        

    # then merge edges

    newEdges = list()

    for g in groupsDict.keys():

        neighbors = [ (u,d) for sp in groupsDict[g] for u,d in scn[sp].items() ]

        neighbors_dict =dict()
        for n,data in neighbors:
            neighbors_dict[n] = neighbors_dict.get(n,{'weight':0})
            neighbors_dict[n]['weight'] += data['weight']

        newEdges += [ (g,v,data) for v,data in neighbors_dict.items() ]

    
    res = nx.Graph()
    res.add_edges_from(newEdges)
    res.add_nodes_from(newNodes)
    
    for n,d in newNodes.items():
        res.node[n].update(**d)
        
    return res
    

In [ ]:
g = aggregateByFamily(scn)
g.nodes(data=True)

In [ ]:
list(newNodes.items())

In [ ]:
list(zip(*newNodes))

In [ ]:
newNodes

In [ ]:
g = aggregateByFamily(scn)

g.nodes(data=True)

In [ ]:
newNodes

In [ ]:
spNodes = ( (n,d) for n,d in scn.nodes(data=True) if d['bipartite']==1 )

sum(  v['degree_weighted'] for k,v in spNodes)

In [ ]:
spNodes = ( (n,d) for n,d in scn.nodes(data=True) if d['bipartite']==1 )

sum(  v['degree_weighted'] for k,v in fDict.items())

In [ ]:
d = {'a':1,'b':2}

d

In [ ]:
from copy import deepcopy

In [ ]:
scn_aggregate=deepcopy(scn)

In [ ]:
nodes = [ (d['family'],d) for n,d in scn.nodes(data=True) if d['bipartite']==1 ]

In [ ]:
nodes

In [ ]:
scn.nodes(data=True)

In [ ]:
occs[['scientificName','family']]

In [ ]:
fcn = SpeciesCollectorsNetwork(species=occs_sporg['family'],
                               collectorsNames=occs_sporg['recordedBy_atomized'],
                               weighted=True, namesMap=nm)

In [ ]:
nodesToRemove = ['etal','']
fcn.remove_nodes_from(nodesToRemove)

# nodes degrees
nx.set_node_attributes(fcn, 'degree', fcn.degree())

# nodes weighted degrees
nx.set_node_attributes(fcn, 'degree_weighted', fcn.degree(weight='weight'))

In [ ]:
nx.write_gexf(fcn, './ub_graph_fcn.gexf')

# Exploration

#### Who are the most prolific collectors?

In [ ]:
rc_dict = {
    'axes.grid':True,
    'grid.color':'.9'
}

sns.set_context('talk')
sns.set_style('ticks', rc_dict)


In [ ]:
f,(ax1,ax2) = plt.subplots(2,1,sharex=True,figsize=(8,12))

# ax1
d = [ data['tot_records'] for name,data in cwn.nodes(data=True) ]
y,x = np.histogram(d,bins=range(1,20141,1),density=True)
y = np.cumsum(y)

y_norm,x_norm = np.histogram(np.random.normal(loc=10000, scale=3000, size=20140),
             bins=range(1,20141,1), density=True)
y_norm = np.cumsum(y_norm)

ax1.plot(x[:-1],y)
ax1.plot(x_norm[:-1],y_norm,ls='dashed')
ax1.semilogx()

plt.xlabel("# Records")
ax1.set_ylabel("Cumulative percentual of collectors")

# ax2

from collections import Counter
numCols = len(d)
x,y = list(zip(*[ (nrecs,c) for nrecs,c in Counter(d).items() ]))

ax2.scatter(x,y,facecolor='none',edgecolors='k',marker='o',s=80, linewidth=0.2)
ax2.loglog()

ax2.set_ylabel("# Collectors")

---

In [ ]:
occs['num_collabs'] = occs['recordedBy_atomized'].apply(lambda l: len(l)-1 )

In [ ]:
num_collabs = [ (n,occs['num_collabs'].loc[ni[n]].mean()) for n in cwn.nodes() ]

In [ ]:
[ (u,v) for u,v in num_collabs if u=='proenca,ceb' ]

In [ ]:
plt.hist( [m for n,m in num_collabs],bins=range(12))
plt.xlabel("Average # of Collaborators")
plt.ylabel("# Collectors")

In [ ]:
sns.distplot([m for n,m in num_collabs],bins=range(12),kde_kws={'bw':0.5})
plt.xlabel("Average # of Collaborators")
plt.ylabel("Percentage of Collectors")

**Figure. Distribution of the average number of collaborators for each collector.**

---

In [ ]:
from scipy.optimize import curve_fit
y,x = np.histogram(occs['num_collabs']+1,bins=range(1,11))

func_powerlaw = lambda x,a,b0,b1: a*pow(x,b0)*np.exp(-x/b1) # power law w/ exponential cutoff

pars_powerlaw,covs_powerlaw = curve_fit(func_powerlaw,x[:-1],y, maxfev=100000)


plt.scatter(x[:-1],y,facecolor='none',edgecolor='k',linewidth=1,s=400)
plt.plot(x,func_powerlaw(x,*pars_powerlaw), '--')

plt.yscale('log')
plt.xscale('log')

plt.ylim(0.5e0,2e5)
plt.xlabel("# Collectors")
plt.ylabel("# Records")

**Figure. Number of records by size of the collectors team. **

---

## Should the number of species collected follow a power law?

In [ ]:
from collections import Counter
collectors = nx.bipartite.sets(scn)[0]

degree_dist = Counter(scn.degree(collectors).values())

In [ ]:
degree_dist = Counter( d['degree_weighted'] for n,d in scn.nodes(data=True) if d['bipartite']==0)

In [ ]:
degrees,cnts = zip(*degree_dist.items())

In [ ]:
plt.scatter(degrees,cnts,facecolor='none',edgecolor='k')

plt.scatter(X,func(X,*params),marker='+',edgecolor='k', linewidth=0.5)

plt.xscale('log')
plt.yscale('log')

In [ ]:
from scipy.optimize import curve_fit

func = lambda x,l,t,a: (l**(1-a))/(t*(1-a))*x**(-a)*np.exp(-l*x) 

X = degrees
y = cnts

params,covs = curve_fit(func,X,y)
params

In [ ]:
func(X,*params)

In [ ]:
plt.plot(X,func(X,*params))

In [ ]:
sig1 = [0,2,0,0,3,41,54,0,3,13,4,5,0,0,1,0,0,1,0,3]
sig2 = [32,0,2,0,3,0,56,0,13,0,0,42,0,21,0,0,1,0,4,9]

sig1 = [0,0,2,1,0]
sig2 = [0,0,0,0,0]

In [ ]:
np.linalg.norm(sig1)

In [ ]:
print(len(sig1),len(sig2))

In [ ]:
np.dot(sig1,sig2)/(np.linalg.norm(sig1)*np.linalg.norm(sig2))